In [1]:
! pip install sklearn

     |████████████████████████████████| 6.7MB 3.6MB/s eta 0:00:01     |█████████████                   | 2.7MB 3.6MB/s eta 0:00:02
     |████████████████████████████████| 296kB 14.3MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=c6ce9138f231dcab6cc44af654be5cb74dfcb829d839c7bbf48217e7573b3ffa
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You should consider upgrading via the 'pip install --upgrade pip' command.


# Imports

In [2]:
from fastai.text import *

In [3]:
import sys

sys.path.append("../../")
from eval.exp.nb_evaluation import *

sys.path.append("../../../")
from src.proc.exp.nb_proc import *
from src.prep.exp.nb_prep import *

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# setup paths and model type
model_path = Path("/tf/data/models")
data_path  = Path("/tf/data/datasets")

task_type = "merged"

# Load Data

In [5]:
df_trn, df_val, df_tst = read_data(data_path/task_type)

In [7]:
bs = 8

In [8]:
data = gen_lm_data(df_trn, df_val, task_type, data_path, sample = 0.01, bs = bs)
# data.path = str(data_path/task_type)

In [9]:
data.path

PosixPath('/tf/data/datasets')

In [10]:
data.save('data_lm_1pct.pkl')

/usr/local/lib/python3.6/dist-packages/fastai/basic_data.py:152: UserWarning: Serializing the `DataBunch` only works when you created it using the data block API.
  warn("Serializing the `DataBunch` only works when you created it using the data block API.")


In [14]:
str(data_path/task_type/'data_lm_10pct.pkl')

'/tf/data/datasets/merged/data_lm_10pct.pkl'

In [13]:
data = load_data(data_path/task_type, 'data_lm_1pct.pkl', bs=bs)

FileNotFoundError: [Errno 2] No such file or directory: '/tf/data/datasets/merged/data_lm_1pct.pkl'

In [ ]:
data.save('data_lm_10pct.pkl')

In [ ]:
len(data.train_ds), len(data.valid_ds)

# Model Setup

In [ ]:
pretrained = False

In [ ]:
learn = language_model_learner(data, AWD_LSTM, drop_mult = 0.3, pretrained = pretrained)

In [ ]:
learn.lr_find()
learn.recorder.plot()

# Model Training

In [ ]:
# Set hyperparameters
max_lr = 1e-2
moms = (0.5, .75)
pct_strt = 0.02

In [ ]:
learn.fit_one_cycle(5, max_lr)#, moms=moms, pct_start = pct_strt)

In [ ]:
learn.save(model_path/'awd_lstm_do_30pct')

In [ ]:
learn.load(model_path/'awd_lstm_do_30pct')

In [ ]:
learn.export(file = model_path/'awd_lstm_do_30pc.pkl')

In [ ]:
learn = load_model(df_trn, file = model_path/'awd_lstm_do_30pc.pkl')

In [ ]:
learn.recorder.plot_losses()

# Model Evaluation

In [ ]:
print(df_val["query"][50000])
print("\n\n" + df_val["res"][50000])

In [ ]:
print(df_val["query"][75_000])
print("\n\n" + df_val["res"][75_000])

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load(str(data_path/"merged/model.model"))

In [ ]:
TEXT = df_val["query"][50_000]
N_WORDS = 200
N_SENTENCES = 2

In [ ]:
print(TEXT)

In [ ]:
print(sp.DecodePieces(learn.predict(TEXT, 100, temperature=0.75).split(" ")))

# Evaluation

In [ ]:
learn.predict??

In [ ]:
# Grabs entire model's response up until special xxbos token,
# i.e. once model begins a new sentence we consider the model finished with its answer.
def get_res(mdl, inpt, n_toks = 1_000):
    res = mdl.predict(inpt, n_toks, temperature=0.75).split(" ")
    res = sp.DecodePieces(res).split(" ")
    try:
        end_res = res.index("xxbos")
    except:
        end_res = len(res) - 1
    
    res = " ".join(res[:end_res])[len(inpt.replace(" ", '')):]
    
    return res

In [ ]:
res = get_res(learn, "public static void main() {return;}<$bug$>", n_toks = 10)
res # [0:len("public static void main() {return;}<$bug$>")]

In [ ]:
# Evaluation metrics for vulnerability detection - Accuracy, Precision, Recall
def eval_vuln(mdl, tst):
    tps, tns, fps, fns = 0, 0, 0, 0
    for inpt, lbl in zip(tst["query"], tst["res"]):
        pred = get_res(mdl, inpt, n_toks = 10)
        if lbl == "yes":
            if pred == lbl:
                tps += 1
            else: fns += 1
        else:
            if pred == lbl:
                tns += 1
            else: fps += 1
            
    acc   = (tps + tns) / len(tst)
    prec  = tps / (tps + fps) if (tps + fps) != 0 else 0.
    recal = tps / (tps + fns) if (tps + fns) != 0 else 0.
    
    return acc, prec, recal

In [ ]:
acc, prec, recal = eval_vuln(learn, df_val[:300])

In [ ]:
acc, prec, recal

In [ ]:
len(df_val[:10])